# Problem Satement

The goal is to help The Department of Housing and Development manage 311 complaints that are not necessarily urgent. Large volumes of complaints are impacting the overall operations of the agency.

The agency is looking to manage the large volume of complaints by answering the following quastions:

1. Which type of complaint should be focused on 
2. Are there any particular boroughs that should be focused on due to this type of complaint
3. Does the complaint type have any obvious relationships with characteristics of the buildings
4. can we build a predictive model to be built for future prediciton of the complaint type

In [ ]:
#if Folium Libraries don't Load
# """!conda install -c conda-forge folium=0.11.0 --yes """

In [ ]:
# !pip install plotly.express

In [1]:
$THEANO_FLAGS=mode=FAST_RUN,device=gpu0,floatX=float32 ipython notebook

SyntaxError: invalid syntax (<ipython-input-1-57dd0c5068d9>, line 1)

In [1]:
import pandas as pd
import json
import math
import plotly.express as px
import numpy as np
from sklearn import neighbors

from folium import plugins

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

## Load Data

In [4]:
Orig_HBD = pd.read_csv("311_Service_Requests_from_2010_to_Present_min.csv")

In [5]:
Orig_HBD.shape

(6019843, 16)

We can see there is 6,019,843 samples and 16 features or columns

### Column Descriptions

**Uunique Key:** Uniques identifier of Service Requests(SR) in open dataset

**Created Date:** Date SR was requested

**Closed Date:** Date SR was closed by responding agency

**Complaint Type:** This is the first lvl of hierarchy identifying the topic of the incident or condition. May have corresponding descriptor or not.

**Location Type:** Describes the type of location used in the address information.

**Incident Zip:** Zip code, provided by geo validation.

**Incident Address:** House number of incident address provided by submitter.

**Street Name:** Street name of incident.

**Address Type:** The type of incident location information available.

**City:** City of the incident.

**Status:** Status of SR submitted.

**Resolution Description:** Describes last action taken on the SR by the responding agency. May describe next or future steps.

**Borough:** Self described.

**Latitude:** Self described.

**Longitiude:** Self described.

In [6]:
Orig_HBD.head()

Unnamed: 0  Unique Key            Created Date Closed Date  \
0           0    45531130  02/02/2020 06:09:17 AM         NaN   
1           1    45529784  02/02/2020 02:15:24 PM         NaN   
2           2    45527528  02/02/2020 02:27:41 AM         NaN   
3           3    45530329  02/02/2020 12:13:18 PM         NaN   
4           4    45528814  02/02/2020 01:59:44 PM         NaN   

         Complaint Type         Location Type  Incident Zip  \
0        HEAT/HOT WATER  RESIDENTIAL BUILDING       10019.0   
1  UNSANITARY CONDITION  RESIDENTIAL BUILDING       11204.0   
2        HEAT/HOT WATER  RESIDENTIAL BUILDING       11372.0   
3        HEAT/HOT WATER  RESIDENTIAL BUILDING       10458.0   
4             APPLIANCE  RESIDENTIAL BUILDING       11209.0   

          Incident Address         Street Name Address Type             City  \
0     426 WEST   52 STREET    WEST   52 STREET      ADDRESS         NEW YORK   
1           1751 67 STREET           67 STREET      ADDRESS         BROOKLYN   
2          87-15 37 AVENUE           37 AVENUE      ADDRESS  Jackson Heights   
3  2405 SOUTHERN BOULEVARD  SOUTHERN BOULEVARD      ADDRESS            BRONX   
4            223 78 STREET           78 STREET      ADDRESS         BROOKLYN   

  Status                             Resolution Description    Borough  \
0   Open  The following complaint conditions are still o...  MANHATTAN   
1   Open  The following complaint conditions are still o...   BROOKLYN   
2   Open  The following complaint conditions are still o...     QUEENS   
3   Open  The following complaint conditions are still o...      BRONX   
4   Open  The following complaint conditions are still o...   BROOKLYN   

    Latitude  Longitude  
0  40.765132 -73.988993  
1  40.618484 -73.992673  
2  40.750269 -73.879432  
3  40.853773 -73.881558  
4  40.629745 -74.030533

Off the bat we saee there are some missing data in the `Closed Date` column.

In [21]:
Na = Orig_HBD.isnull().sum()
Na

Unnamed: 0                     0
Unique Key                     0
Created Date                   0
Closed Date               126657
Complaint Type                 0
Location Type              52824
Incident Zip               80697
Incident Address           52825
Street Name                52825
Address Type               84765
City                       80274
Status                         0
Resolution Description      7826
Borough                        0
Latitude                   80671
Longitude                  80671
dtype: int64

We will come back to the missing Data and right now focus on the two Features that are the focus of our attention: `Complaint Type` and `Borough` which have no missing data.

## Data PreProcessing and Selection

I'm going to create another dataframe with the columns the will be usefull in determining the Answer to Question 1: Which Complaint Type should the Agency focus on first.

But first I'm going to rename the columns for simplicity sake.

In [ ]:
HBD = Orig_HBD.rename(columns={'Complaint Type':'Complaint_Type', 'Incident Zip':'Incident_Zip',
                               'Unique Key':'Unique_Key', 'Created Date':'Created_Date','Closed Date':'Closed_Date',
                               'Location Type':'Location_Type','Address Type':'Address_Type', 'Street Name':'Street_Name'})
HBD.head()